In [1]:
from module_Plot import Plotting

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors

import numpy as np
import pandas as pd
import os
import copy
import h5py

plt.rcParams['axes.unicode_minus'] = False


In [2]:
sub_dir = 'used_data'
loc_main = r'E:\data\_results\common_radar'
fn_csv = os.path.join(loc_main, sub_dir, 'TG_2D_rain_V06A.csv')
fn_TRMMh5 = os.path.join(loc_main, sub_dir, 'TRMMKu_profile_rain_V06A.h5')
fn_GPMh5 = os.path.join(loc_main, sub_dir, 'GPMKu_profile_rain_V06A.h5')

fig_idxs = [f'({chr(ii)})' for ii in range(97, 123)]
linestyles = ['-', '--', ':', '-.']

add_text = False

In [3]:
df_TG = pd.read_csv(fn_csv, compression='gzip')
h5_pro = {'TRMM': fn_TRMMh5, 'GPM': fn_GPMh5}

da_pro = {}
for key, fnh5 in h5_pro.items():
    hold = h5py.File(fnh5, 'r')
    da_pro[key] = hold[os.path.split(fnh5)[-1].split('_V')[0]][:]
    da_pro[key][np.isnan(da_pro[key])] = 0
    hold.close()
print(df_TG.shape, [(key, val.shape) for key, val in da_pro.items()])
print(da_pro['GPM'].min(), da_pro['TRMM'].min())
print(df_TG.columns)

(217618, 35) [('TRMM', (217618, 176)), ('GPM', (217618, 176))]
0.0 0.0
Index(['Time_TRMMKu', 'Time_GPMKu', 'dlt_Dm', 'dlt_Tsec', 'orbitID_TRMMKu',
       'loc_x_TRMMKu', 'loc_y_TRMMKu', 'orbitID_GPMKu', 'loc_x_GPMKu',
       'loc_y_GPMKu', 'Lon_TRMMKu', 'Lat_TRMMKu', 'LocalHour_TRMMKu',
       'nearSurfRain_TRMMKu', 'dBZ20_Htop_TRMMKu', 'dBZ35_Htop_TRMMKu',
       'rainType_TRMMKu', 'HBB_TRMMKu', 'Lon_GPMKu', 'Lat_GPMKu',
       'LocalHour_GPMKu', 'nearSurfRain_GPMKu', 'dBZ20_Htop_GPMKu',
       'dBZ35_Htop_GPMKu', 'LandOcean_GPMKu', 'rainType_GPMKu', 'HBB_GPMKu',
       'nearSurfdBZ_TRMMKu', 'surfRainPhase_TRMMKu', 'nearSurfdBZ_GPMKu',
       'surfRainPhase_GPMKu', 'dBZ20_Npxl_TRMMKu', 'dBZ35_Npxl_TRMMKu',
       'dBZ20_Npxl_GPMKu', 'dBZ35_Npxl_GPMKu'],
      dtype='object')


In [18]:
levs = np.array([1,5]+list(np.arange(10,101, 5)))  # percentage
cb_labs = [1] + list(np.arange(10, 101, 10))
bw = 0.5 # degree
bins = [np.arange(-20, 20.000001, bw), np.arange(0, 16.01, 0.25)]
bins_L =[[14, 50]]
color_ax2  = 'purple'
self_colors = ['gold','green','blue','red','purple']
########
def plot_pixel_dlt_pct_profile(fn, df, df_L, xykeys, hue='level_1', xykeys_L=['dBZ', 'Height'], ipanel=-1):
    panel0, panel1 = False, False
    if ipanel in [-1, 0]:
        panel0 = True
    elif ipanel in [-1, 1]:
        panel1 = True
    
    plot = Plotting(global_xy=False, fontsize=14)
    cmap = copy.copy((cm.get_cmap('rainbow')))    
    
    sns.set_context('talk', font_scale=1)
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6), sharey=True)
    fig.subplots_adjust(wspace=0.11, hspace=0.15)
    
    ## ax[0]
    X0, Y0 = np.array(0), np.array(0)
    if panel1:
        ii = 1
        grid, Xedge, Yedge, _ = ax[ii].hist2d(df[xykeys[0]], df[xykeys[1]], bins=bins, visible=False) #

        maxDA = int(grid.max())
        print('Max_grid:', maxDA)
        pctDA = 100*grid.T/maxDA

        obj = ax[ii].pcolormesh(Xedge, Yedge, pctDA,
                      cmap=cmap, shading='auto', 
                      norm=colors.BoundaryNorm(boundaries=levs, ncolors=cmap.N), 
                      edgecolors='none', alpha=1, zorder=2)
        obj.cmap.set_under('white')
        obj.cmap.set_over('firebrick')

        ax[ii].set_xlim(bins[0].min(), bins[0].max())   
        ax[ii].set_ylim(bins[1].min(), bins[1].max())   

        ax[ii].xaxis.set_major_locator(ticker.MultipleLocator(5))
        ax[ii].xaxis.set_minor_locator(ticker.MultipleLocator(1))
        ax[ii].yaxis.set_major_locator(ticker.MultipleLocator(2))
        ax[ii].yaxis.set_minor_locator(ticker.MultipleLocator(0.5))

        ax[ii].set_xlabel(r'$\Delta$Reflectivity (dBZ)')
    #    ax[ii].set_ylabel('Height (km)')

        cb_ax = ax[ii].inset_axes(bounds=[1.14, 0, 0.04, 0.98])
        cb = fig.colorbar(obj, cb_ax, orientation='vertical') #'horizontal'
        cb.set_ticks(cb_labs)
        cb.set_ticklabels(cb_labs)
        cb.ax.tick_params(direction='in', length=4)
        cb.ax.tick_params(which='minor', direction='in', length=2)
        cb.set_label(' Percent (%)', labelpad=1) #, fontdict={'fontsize': plot.fontsize}

        obj0 = sns.lineplot(data=df, x=xykeys[1], y=xykeys[0], 
                     ax=ax[ii], ci=None, visible=False, legend=False)
        Y0, X0 = obj0.lines[0].get_data()
        TF_idx = np.where(Y0<12.5)    
        ax[ii].plot(X0[TF_idx], Y0[TF_idx], color='k', linestyle='--', lw=3)

        ax[ii].hlines(5.1875, xmin=bins[0].min(), xmax=bins[0].max(), color='k', linestyle=':', lw=2)
        ax[ii].vlines(0, ymin=bins[1].min(), ymax=bins[1].max(), color='k', linestyle=':', lw=2)

        ax2 = ax[ii].twinx()
        sns.kdeplot(data=df, x=xykeys[0], bw_method=bw, ax=ax2, color=color_ax2, linewidth=4)
        ax2.yaxis.set_minor_locator(ticker.MultipleLocator(0.01))
        ax2.tick_params(axis='y', which='major', color=color_ax2, labelcolor=color_ax2)

        ax2.text(x=0.98, y=0.98, s=f'Mean = {df[xykeys[0]].mean():.02f}\nSTD = {df[xykeys[0]].std():.02f}', 
                 ma='right', ha='right', va='top', color=color_ax2, fontsize=16, transform=ax2.transAxes) #

        yticks = ax2.get_yticks() 
        ax2.set_yticks(yticks)
        ax2.set_yticklabels([f'{int(tick*100)}' for tick in yticks])
        ax2.set_ylim([0, 0.095])

        ax2.set_ylabel('Sample density (%)', color=color_ax2)
        ax2.spines['right'].set_color('none')
        ax[ii].spines['right'].set_color(color_ax2)

        ax[ii].text(x=0.01, y=0.98, s=f'{fig_idxs[ii]}', 
                     ma='left', ha='left', va='top', color='black', transform=ax[ii].transAxes)

    ### ax[1]
    if panel0:
        ii = 0
        for key, ls in zip(sorted(set(df_L.keys())), linestyles[:df_L.__len__()]):
            for idx, var in enumerate(df_L[key][hue].unique()):
                data = df_L[key].query(f'{hue}==@var')
                if key =='A': 
                    lab = ' '
                else:
                    lab = var.split('_')[0]
                ax[ii].plot(data[xykeys_L[0]], data[xykeys_L[1]], color=self_colors[idx], linestyle=ls, linewidth=1, label=lab)
        ax[ii].legend(fontsize=14, loc='upper right', bbox_to_anchor=(1, 0.95), 
                      ncol=2, title='TRMM   GPM          ', title_fontsize=14, columnspacing=0.8)

        ax[ii].set_xlim(bins_L[0])
        ax[ii].set_ylim(bins[1].min(), bins[1].max())

        ax[ii].set_xlabel('Reflectivity (dBZ)')
        ax[ii].set_ylabel('Height (km)')

        ax[ii].xaxis.set_major_locator(ticker.MultipleLocator(5))
        ax[ii].xaxis.set_minor_locator(ticker.MultipleLocator(1))
        ax[ii].yaxis.set_major_locator(ticker.MultipleLocator(1))
        ax[ii].yaxis.set_minor_locator(ticker.MultipleLocator(0.25))

        ax[ii].text(x=0.01, y=0.98, s=f'{fig_idxs[ii]}', 
                     ma='left', ha='left', va='top', color='black', transform=ax[ii].transAxes)
    
    plot._save_fig(fn, fig)
    del plot
    return pd.DataFrame({'Height': Y0, 'dBZ_GmT': X0})


titles = ['ALL', 'Convective', 'Stratiform']
sfcType = {0: 'Ocean', 1: 'Land'}
def plot_pixel_pct_profile(fn, df_L, xykeys, hue='level_1', xykeys_L=['dBZ', 'Height']):    
    plot = Plotting(global_xy=False, fontsize=14)
    cmap = copy.copy((cm.get_cmap('rainbow')))    
    
    sns.set_context('talk', font_scale=1)
    nrows, ncols =2, 3
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 12), sharex=True, sharey=True)
    fig.subplots_adjust(wspace=0.15, hspace=0.15)
    
    for rdx in range(nrows):
        for cdx in range(ncols):
            iax = ax[rdx][cdx]
            index = rdx*ncols+cdx
            for key, ls in zip(sorted(set(df_L[index].keys())), linestyles[:df_L[index].__len__()]):
                for idx, var in enumerate(df_L[index][key][hue].unique()):
                    data = df_L[index][key].query(f'{hue}==@var')
                    if key =='A': 
                        lab = ' '
                    else:
                        lab = var.split('_')[0]
                    iax.plot(data[xykeys_L[0]], data[xykeys_L[1]], color=self_colors[idx], linestyle=ls, linewidth=1, label=lab)
            if (rdx==0) & (cdx == 0):
                iax.legend(fontsize=14, loc='upper right', #bbox_to_anchor=(1, 0.95), 
                              ncol=2, title='TRMM   GPM          ', title_fontsize=14, columnspacing=0.8)
                iax.text(x=-0.2, y=0.1, s='Height (km)', rotation=90, 
                             ma='left', ha='left', va='top', color='black', transform=iax.transAxes)
#                iax.set_ylabel('Height (km)')
            else:
                iax.set_ylabel('')

            if (rdx==1) & (cdx == 1):
                iax.set_xlabel('Reflectivity (dBZ)') 
            else:
                iax.set_xlabel('')
            if rdx == 0:
                iax.set_title(titles[cdx])
            iax.set_xlim(bins_L[0])
            iax.set_ylim(bins[1].min(), bins[1].max())

            iax.xaxis.set_major_locator(ticker.MultipleLocator(5))
            iax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
            iax.yaxis.set_major_locator(ticker.MultipleLocator(2))
            iax.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))

            iax.text(x=0.01, y=0.98, s=f'{fig_idxs[rdx*ncols+cdx]}', 
                             ma='left', ha='left', va='top', color='black', transform=iax.transAxes)
            if cdx == 2:
                iax.text(x=1.02, y=0.5,  # x=1.12  1.02
                     s=f'{sfcType[1-rdx]}', rotation=90, 
                     ma='left', ha='left', va='center', color='black', transform=iax.transAxes)
    
    plot._save_fig(fn, fig)
    del plot
    return 

In [20]:
# data-preparation for dlt
hgt = {'dH': 0.125, 'HGT': np.arange(0., 22., 0.125)[::-1].astype(np.float32)}
tmp_hgt = hgt['HGT'][np.newaxis,:]*np.ones(df_TG.shape[0])[:, np.newaxis]
thh_dBZ_GPM, thh_dBZ_TRMM = 13, 18  # dBZ

valid_idxs = df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})&(LandOcean_GPMKu==1)').index.values
#np.where(np.logical_and(df_TG['nearSurfdBZ_GPMKu'] >= thh_dBZ_GPM, df_TG['nearSurfdBZ_TRMMKu'] >= thh_dBZ_TRMM))[0]
df_grp_l = pd.DataFrame({
    'GPM': da_pro['GPM'][valid_idxs].ravel().astype(np.float32),
    'TRMM': da_pro['TRMM'][valid_idxs].ravel().astype(np.float32),
    'dBZ_GmT': da_pro['GPM'][valid_idxs].ravel().astype(np.float32) - da_pro['TRMM'][valid_idxs].ravel().astype(np.float32),
    'Height': tmp_hgt[valid_idxs].ravel(),
#    'LandOcean': (np.ones(hgt['HGT'].shape[0])[np.newaxis,:]*df_TG.loc[valid_idxs, 'LandOcean_GPMKu'].values[:, np.newaxis]).ravel().astype(np.int32)
}).reset_index(drop=True)


strat_idxs = df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})&(rainType_GPMKu==1)&(LandOcean_GPMKu==1)').index.values # stratiform
df_S_l = pd.DataFrame({
    'GPM': da_pro['GPM'][strat_idxs].ravel().astype(np.float32),
    'TRMM': da_pro['TRMM'][strat_idxs].ravel().astype(np.float32),
    'dBZ_GmT': da_pro['GPM'][strat_idxs].ravel().astype(np.float32) - da_pro['TRMM'][strat_idxs].ravel().astype(np.float32),
    'Height': tmp_hgt[strat_idxs].ravel(),
#    'LandOcean': (np.ones(hgt['HGT'].shape[0])[np.newaxis,:]*df_TG.loc[strat_idxs, 'LandOcean_GPMKu'].values[:, np.newaxis]).ravel().astype(np.int32)
}).reset_index(drop=True)

conv_idxs = df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})&(rainType_GPMKu==2)&(LandOcean_GPMKu==1)').index.values # convective
df_C_l = pd.DataFrame({
    'GPM': da_pro['GPM'][conv_idxs].ravel().astype(np.float32),
    'TRMM': da_pro['TRMM'][conv_idxs].ravel().astype(np.float32),
    'dBZ_GmT': da_pro['GPM'][conv_idxs].ravel().astype(np.float32) - da_pro['TRMM'][conv_idxs].ravel().astype(np.float32),
    'Height': tmp_hgt[conv_idxs].ravel(),
#    'LandOcean': (np.ones(hgt['HGT'].shape[0])[np.newaxis,:]*df_TG.loc[conv_idxs, 'LandOcean_GPMKu'].values[:, np.newaxis]).ravel().astype(np.int32)
}).reset_index(drop=True)

print(valid_idxs.shape, strat_idxs.shape, conv_idxs.shape)

valid_idxs = df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})&(LandOcean_GPMKu==0)').index.values
#np.where(np.logical_and(df_TG['nearSurfdBZ_GPMKu'] >= thh_dBZ_GPM, df_TG['nearSurfdBZ_TRMMKu'] >= thh_dBZ_TRMM))[0]
df_grp_o = pd.DataFrame({
    'GPM': da_pro['GPM'][valid_idxs].ravel().astype(np.float32),
    'TRMM': da_pro['TRMM'][valid_idxs].ravel().astype(np.float32),
    'dBZ_GmT': da_pro['GPM'][valid_idxs].ravel().astype(np.float32) - da_pro['TRMM'][valid_idxs].ravel().astype(np.float32),
    'Height': tmp_hgt[valid_idxs].ravel(),
#    'LandOcean': (np.ones(hgt['HGT'].shape[0])[np.newaxis,:]*df_TG.loc[valid_idxs, 'LandOcean_GPMKu'].values[:, np.newaxis]).ravel().astype(np.int32)
}).reset_index(drop=True)


strat_idxs = df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})&(rainType_GPMKu==1)&(LandOcean_GPMKu==0)').index.values # stratiform
df_S_o = pd.DataFrame({
    'GPM': da_pro['GPM'][strat_idxs].ravel().astype(np.float32),
    'TRMM': da_pro['TRMM'][strat_idxs].ravel().astype(np.float32),
    'dBZ_GmT': da_pro['GPM'][strat_idxs].ravel().astype(np.float32) - da_pro['TRMM'][strat_idxs].ravel().astype(np.float32),
    'Height': tmp_hgt[strat_idxs].ravel(),
#    'LandOcean': (np.ones(hgt['HGT'].shape[0])[np.newaxis,:]*df_TG.loc[strat_idxs, 'LandOcean_GPMKu'].values[:, np.newaxis]).ravel().astype(np.int32)
}).reset_index(drop=True)

conv_idxs = df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})&(rainType_GPMKu==2)&(LandOcean_GPMKu==0)').index.values # convective
df_C_o = pd.DataFrame({
    'GPM': da_pro['GPM'][conv_idxs].ravel().astype(np.float32),
    'TRMM': da_pro['TRMM'][conv_idxs].ravel().astype(np.float32),
    'dBZ_GmT': da_pro['GPM'][conv_idxs].ravel().astype(np.float32) - da_pro['TRMM'][conv_idxs].ravel().astype(np.float32),
    'Height': tmp_hgt[conv_idxs].ravel(),
#    'LandOcean': (np.ones(hgt['HGT'].shape[0])[np.newaxis,:]*df_TG.loc[conv_idxs, 'LandOcean_GPMKu'].values[:, np.newaxis]).ravel().astype(np.int32)
}).reset_index(drop=True)


print(valid_idxs.shape, strat_idxs.shape, conv_idxs.shape)

(27486,) (19036,) (8364,)
(81373,) (53246,) (27800,)


In [21]:
df_TG.query(f'(nearSurfdBZ_GPMKu>={thh_dBZ_GPM})&(nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM})').shape[0]

110783

In [22]:
# data-preparation for pct
'''
df_grp['Height250'] = df_grp.loc[:, 'Height']*100
df_grp['Height250'] = df_grp['Height250'].apply(lambda v: v if v%1 == 0 else v-12.5)
df_grp['Height250'] = df_grp['Height250']/100
'''
#########
hgtCol = 'Height'
percentage = [0.25,0.5,0.75,0.95]
PCT_dBZ = [{}, {}, {}, {}, {}, {}]

for idx, idf in enumerate([df_grp_l, df_C_l, df_S_l, df_grp_o, df_C_o, df_S_o]):
    tmpdf = idf.groupby(hgtCol)
    PCT_dBZ[idx]['A'] = tmpdf['TRMM'].quantile(q=percentage).reset_index()
    PCT_dBZ[idx]['A']['level_1'] = (PCT_dBZ[idx]['A']['level_1']*100).apply(lambda v: f'{v:.0f}%_TRMM')
    PCT_dBZ[idx]['A'].rename(columns={'TRMM': 'dBZ'}, inplace=True)
    PCT_dBZ[idx]['A'].loc[PCT_dBZ[idx]['A']['dBZ']==0, 'dBZ'] = np.nan

    PCT_dBZ[idx]['B'] = tmpdf['GPM'].quantile(q=percentage).reset_index()
    PCT_dBZ[idx]['B']['level_1'] = (PCT_dBZ[idx]['B']['level_1']*100).apply(lambda v: f'{v:.0f}%_GPM')
    PCT_dBZ[idx]['B'].rename(columns={'GPM': 'dBZ'}, inplace=True)
    PCT_dBZ[idx]['B'].loc[PCT_dBZ[idx]['B']['dBZ']==0, 'dBZ'] = np.nan
    
    del tmpdf
#####################

In [23]:
name = 'stat_GmT_dBZ_profiles'
'''
resdf = plot_pixel_dlt_pct_profile(name, df_grp, PCT_dBZ, xykeys=['dBZ_GmT', 'Height'], 
                                   hue='level_1', xykeys_L=['dBZ', hgtCol], ipanel=0)
pd.set_option('display.max_rows', None)
resdf
'''
plot_pixel_pct_profile(name, PCT_dBZ, xykeys=['dBZ_GmT', 'Height'], 
                                   hue='level_1', xykeys_L=['dBZ', hgtCol])
